In [1]:
import sys
sys.path.append("../")

In [2]:
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from pyspark import  SparkContext
from pyspark.sql import SparkSession
from ta import add_all_ta_features
from ta.utils import dropna
from utils.draw_candle_image import *
from query_dataset import *
from generate_dataset_tool.tool_v4 import (
    generate_three_image_of_input_with_ema_9_macd_histogram,
    save_to_tensorflow_dataset_with_ema_9_macd_histogram,
)
import json
import keras
import numpy as np
import numpy as np
import os
import pandas as pd
import random
import ta
import ta.momentum
import ta.trend
import ta.trend
import ta.volatility
import tensorflow as tf
from tabulate import tabulate

2024-12-11 17:17:36.813624: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 17:17:37.082611: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 17:17:37.163866: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 17:17:37.186123: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 17:17:37.389718: I tensorflow/core/platform/cpu_feature_guar

In [3]:
base_query = get_base_query()
save_to_tensorflow_dataset_func = save_to_tensorflow_dataset_with_ema_9_macd_histogram
generate_image_func = generate_three_image_of_input_with_ema_9_macd_histogram

In [4]:
symbol = input("Symbol = ").strip().lower()
interval = input("interval = ").lower()
file_csv = f'../data/{symbol}/{symbol}_to_now_{interval}.csv'
stock_df = pd.read_csv(file_csv)
stock_df[Date] = pd.to_datetime(stock_df[Date])
stock_df[Date] = stock_df[Date].dt.tz_localize(None)
stock_df.sort_values(by=Date, ascending=True, inplace=True)
stock_df.reset_index(drop=True, inplace=True)

stock_df[Macd] = ta.trend.macd(stock_df[Close], fillna=True)
stock_df[Macd_signal]= ta.trend.macd_signal(stock_df[Close], fillna=True)
stock_df[Macd_histogram] = ta.trend.macd_diff(stock_df[Close], fillna=True)

indicator_bb = ta.volatility.BollingerBands(close=stock_df[Close], window=20, window_dev=2, fillna=True)
stock_df[BB_avg] = indicator_bb.bollinger_mavg()
stock_df[BB_high] = indicator_bb.bollinger_hband()
stock_df[BB_low] = indicator_bb.bollinger_lband()

indicator_SMA = ta.trend.SMAIndicator(stock_df[Close], 20, fillna=True)
stock_df[SMA] = indicator_SMA.sma_indicator()

indicator_EMA_50 = ta.trend.EMAIndicator(stock_df[Close], 50, fillna=True)
stock_df[EMA_50] = indicator_EMA_50.ema_indicator()

indicator_EMA_9 = ta.trend.EMAIndicator(stock_df[Close], 9, fillna=True)
stock_df[EMA_9] = indicator_EMA_9.ema_indicator()

indicator_EMA_200 = ta.trend.EMAIndicator(stock_df[Close], 200, fillna=True)
stock_df[EMA_200] = indicator_EMA_200.ema_indicator()

In [5]:
stock_df.tail(1)

,Open,High,Low,Close,Volume,Date,Macd,Macd_signal,Macd_histogram,BB_avg,BB_high,BB_low,SMA,EMA_50,EMA_9,EMA_200
3728,96407.99,97836.0,95693.88,97185.17,9115.363698,2024-12-01,5011.965664,5623.285954,-611.32029,93721.8715,100888.3666,86555.3764,93721.8715,82819.783879,95726.795167,68467.005461


In [6]:
years = stock_df[Date].dt.year.unique()

In [7]:
years

array([2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024],
      dtype=int32)

In [8]:
def get_value_of_input(message, function_validate, function_convert, default=None):
    input_str = input(message).strip().lower()
    if not function_validate(input_str):
        return default
    else:
        return function_convert(input_str)

In [9]:
min_year = get_value_of_input(f"Min year = (default = {min(years)})", str.isdigit, int, min(years))
max_year = get_value_of_input(f"Max year = (default = {max(years)})", str.isdigit, int, max(years))

take_years = list(range(min_year, max_year + 1))
years_to_test = dict()
while True:
    year_to_test = input("year to test (also the year for continue learning) = (s for stop)")
    if not str.isdigit(year_to_test):
        break
    else:
        year_to_test = int(year_to_test)
    
    try:
        current_month = datetime.now().month
        from_month = get_value_of_input(f"year = {year_to_test}, from month = (default = 1)", str.isdigit, int, 1)
        to_month = get_value_of_input(f"year to test = {year_to_test}, to month = (default = {current_month})", str.isdigit, int, current_month)
        years_to_test[year_to_test] = list(range(from_month, to_month + 1))
    except Exception as err:
        continue
    
years = take_years
list_years_to_test_str = f"{min(years_to_test)}_{max(years_to_test)}"

In [10]:
figscale = 0.5
previous_days = 0
next_days = 2
days_result = 3
type_dataset = "three_image_ema_macd"
folder_save = f"../dataset/{symbol}_with_{type_dataset}"
if os.path.exists(folder_save):
    print(f"Folder {folder_save} already exists.")
    continue_with_name = input("append name: ")
    folder_save += continue_with_name
    
os.makedirs(f"{folder_save}/model_save", exist_ok=True)
os.makedirs(f"{folder_save}/output_prediction_image", exist_ok=True)

indicator = {
    "show_macd": False,
    "show_macd_signal": False,
    "show_macd_histogram": False,

    "show_BB_avg": False,
    "show_BB_high": False,
    "show_BB_low": False,

    "show_SMA": False,
    "show_EMA_50": False,
    "show_EMA_200": False,
}

Folder ../dataset/btc_with_three_image_ema_macd already exists.


In [11]:
print(f"{min_year=}")
print(f"{max_year=}")
print(f"{years_to_test=}")
print(f"{figscale=}")
print(f"{previous_days=}")
print(f"{next_days=}")
print(f"{days_result=}")
print(f"{type_dataset=}")
print(f"{folder_save=}")

min_year=2014
max_year=2024
years_to_test={2024: [9, 10, 11, 12]}
figscale=0.5
previous_days=0
next_days=2
days_result=3
type_dataset='three_image_ema_macd'
folder_save='../dataset/btc_with_three_image_ema_macd'


In [12]:
# stock_df[stock_df["Date"].dt.year == 2014]

In [13]:
# draw_candle_image(
#     add_days_around_date("2019-09-21", stock_df, previous_days, next_days), 
#     show_x_y=False, 
#     show_volume=False, 
    
#     **indicator,
    
#     figscale=1, 
#     figcolor="black", 
#     preview_image=True,
#     return_image_tensor=False
# )

In [15]:
# os.environ["JAVA_HOME"] = r'C:\Program Files\Java\jre-1.8'
# os.environ["PYSPARK_PYTHON"] = "python" 
spark: SparkSession = SparkSession.builder.master("local").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/12/11 17:17:58 WARN Utils: Your hostname, DESKTOP-64JECLN resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/11 17:17:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/11 17:17:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
data_df_spark = spark.createDataFrame(stock_df)
data_df_spark.createOrReplaceTempView(data_stock)

In [17]:
def random_month_test_in_year(total_years, years_to_test):
    month_test_in_year = dict()
    for year in total_years:
        if year in years_to_test:
            number_month = years_to_test[year]
            month_test_in_year[year] = []
            for _ in range(number_month):
                if year == 2014:
                    month_test = random.randint(9, 12)
                elif year == 2024:
                    month_test = random.randint(1, 11)
                else:
                    month_test = random.randint(1, 12)
                    
                month_test_in_year[year].append(month_test)
        else:
            month_test_in_year[year] = [0]
    return month_test_in_year

In [18]:
def not_random_month_test_in_year(total_years, years_to_test):
    month_test_in_year = dict()
    for year in total_years:
        if year in years_to_test:
            number_month = years_to_test[year]
            month_test_in_year[year] = years_to_test[year]
        else:
            month_test_in_year[year] = [0]
    return month_test_in_year

In [19]:
month_test_in_year = not_random_month_test_in_year(
    years, 
    years_to_test
)

In [20]:
month_test_in_year

{2014: [0],
 2015: [0],
 2016: [0],
 2017: [0],
 2018: [0],
 2019: [0],
 2020: [0],
 2021: [0],
 2022: [0],
 2023: [0],
 2024: [9, 10, 11, 12]}

In [21]:
def generate_exclude_month_query_for_train(year_and_months):
    list_query = []
    for year in year_and_months:
        months = year_and_months[year]
        list_query_of_year = []
        for m in months:
            q = f"(YEAR({Date_normalized}) = {year} AND MONTH({Date_normalized}) != {m})"
            list_query_of_year.append(q)
        query_of_year = " AND ".join(list_query_of_year)
        list_query.append(f"({query_of_year})")
    return " OR ".join(list_query)

In [22]:
def generate_include_month_query_for_test(year_and_months):
    list_query = []
    for year in year_and_months:
        months = year_and_months[year]
        list_query_of_year = []
        for m in months:
            q = f"(YEAR({Date_normalized}) = {year} AND MONTH({Date_normalized}) = {m})"
            list_query_of_year.append(q)
        query_of_year = " OR ".join(list_query_of_year)
        list_query.append(f"({query_of_year})")
    return " OR ".join(list_query)

In [23]:
generate_exclude_month_query_for_train(month_test_in_year)

'((YEAR(Date_normalized) = 2014 AND MONTH(Date_normalized) != 0)) OR ((YEAR(Date_normalized) = 2015 AND MONTH(Date_normalized) != 0)) OR ((YEAR(Date_normalized) = 2016 AND MONTH(Date_normalized) != 0)) OR ((YEAR(Date_normalized) = 2017 AND MONTH(Date_normalized) != 0)) OR ((YEAR(Date_normalized) = 2018 AND MONTH(Date_normalized) != 0)) OR ((YEAR(Date_normalized) = 2019 AND MONTH(Date_normalized) != 0)) OR ((YEAR(Date_normalized) = 2020 AND MONTH(Date_normalized) != 0)) OR ((YEAR(Date_normalized) = 2021 AND MONTH(Date_normalized) != 0)) OR ((YEAR(Date_normalized) = 2022 AND MONTH(Date_normalized) != 0)) OR ((YEAR(Date_normalized) = 2023 AND MONTH(Date_normalized) != 0)) OR ((YEAR(Date_normalized) = 2024 AND MONTH(Date_normalized) != 9) AND (YEAR(Date_normalized) = 2024 AND MONTH(Date_normalized) != 10) AND (YEAR(Date_normalized) = 2024 AND MONTH(Date_normalized) != 11) AND (YEAR(Date_normalized) = 2024 AND MONTH(Date_normalized) != 12))'

In [24]:
generate_include_month_query_for_test(month_test_in_year)

'((YEAR(Date_normalized) = 2014 AND MONTH(Date_normalized) = 0)) OR ((YEAR(Date_normalized) = 2015 AND MONTH(Date_normalized) = 0)) OR ((YEAR(Date_normalized) = 2016 AND MONTH(Date_normalized) = 0)) OR ((YEAR(Date_normalized) = 2017 AND MONTH(Date_normalized) = 0)) OR ((YEAR(Date_normalized) = 2018 AND MONTH(Date_normalized) = 0)) OR ((YEAR(Date_normalized) = 2019 AND MONTH(Date_normalized) = 0)) OR ((YEAR(Date_normalized) = 2020 AND MONTH(Date_normalized) = 0)) OR ((YEAR(Date_normalized) = 2021 AND MONTH(Date_normalized) = 0)) OR ((YEAR(Date_normalized) = 2022 AND MONTH(Date_normalized) = 0)) OR ((YEAR(Date_normalized) = 2023 AND MONTH(Date_normalized) = 0)) OR ((YEAR(Date_normalized) = 2024 AND MONTH(Date_normalized) = 9) OR (YEAR(Date_normalized) = 2024 AND MONTH(Date_normalized) = 10) OR (YEAR(Date_normalized) = 2024 AND MONTH(Date_normalized) = 11) OR (YEAR(Date_normalized) = 2024 AND MONTH(Date_normalized) = 12))'

In [25]:
candle_type = [
    Is_start_bullish_engulfing_candle,
    Is_start_bearish_engulfing_candle,
    Is_star_doji_candle,
    Is_dragonfly_doji_candle,
    Is_gravestone_doji_candle,
    Is_start_morning_star_candle,
    Is_start_evening_star_candle,
    Is_hammer_candle,
    Is_inverted_hammer_candle,
    Is_start_harami_down_candle,
    Is_start_harami_up_candle,
    Is_start_tweezer_top_candle,
    Is_start_tweezer_bottom_candle,
    Is_marubozu_candle,
    Is_spin_candle,
    Is_start_next_3_day_up_candle,
    Is_start_next_3_day_down_candle
]

In [26]:
candle_type_and_directory_save = {
    Is_start_bullish_engulfing_candle: "bullish_engulfing",
    Is_start_bearish_engulfing_candle: "bearish_engulfing",
    Is_star_doji_candle: "star_doji",
    Is_dragonfly_doji_candle: "dragonfly_doji",
    Is_gravestone_doji_candle: "gravestone_doji",
    Is_start_morning_star_candle: "morning_star",
    Is_start_evening_star_candle: "evening_star",
    Is_hammer_candle: "hammer",
    Is_inverted_hammer_candle: "inverted_hammer",
    Is_start_harami_down_candle: "harami_down",
    Is_start_harami_up_candle: "harami_up",
    Is_start_tweezer_top_candle: "tweezer_top",
    Is_start_tweezer_bottom_candle: "tweezer_bottom",
    Is_marubozu_candle: "marubozu",
    Is_spin_candle: "spin",
    Is_start_next_3_day_up_candle: "next_3_day_up",
    Is_start_next_3_day_down_candle: "next_3_day_down",
}

In [27]:
def generate_query(condition_of_time, candle_type_and_directory_save):
    condition_of_candle_type = " OR ".join(candle_type_and_directory_save)
    query = f"""
        {base_query}
        
        SELECT
            COUNT(*) OVER() AS {Total_records},
            *
        FROM {data_stock_base}
        WHERE ({condition_of_candle_type})
        AND ({condition_of_time})
    """
    return query

In [28]:
def save_dataset(
    query,
    candle_type_and_directory_save,
    folder_path
):
    df_spark_all = spark.sql(query)
    df_pandas_all = df_spark_all.toPandas()
    total_records = 0
    
    for candle_type, directory_save in candle_type_and_directory_save.items():
        print(f"{candle_type=}")
        print(f"{directory_save=}")
        
        df_finded = df_pandas_all[df_pandas_all[candle_type]].sort_values(Date_normalized)
        df_finded[Total_records] = len(df_finded)
        print(tabulate(df_finded[0:2], headers='keys', tablefmt='pretty'))
        dataset = save_to_tensorflow_dataset_func(
            df_date=df_finded,
            df_original=stock_df,
            previous_days=previous_days,
            next_days=next_days,
            days_result=days_result,
            number_images_preview=0,
            save_dataset_to_folder=f"{folder_path}/{directory_save}",
            figscale=figscale,
            indicator=indicator,
            function_generate=generate_image_func
        )
        
        if dataset is None:
            dataset = []
        total_records += len(dataset)
    return total_records
    

In [ ]:
# For test
is_run_for_test = input("Run for test? (y/n): ").lower()[0] == "y"
total_records_test = 0
if is_run_for_test:
    query = generate_query(generate_include_month_query_for_test(month_test_in_year), candle_type_and_directory_save)
    total_records_test = save_dataset(query, candle_type_and_directory_save, folder_path=f"{folder_save}/test")
print(f"Total records for test: {total_records_test}")

Total records for test: 0


: 

In [ ]:
# For train
is_run_for_train = input("Run for train? (y/n): ").lower()[0] == "y"
total_records_train = 0
if is_run_for_train:
    query = generate_query(generate_exclude_month_query_for_train(month_test_in_year), candle_type_and_directory_save)
    total_records_train = save_dataset(query, candle_type_and_directory_save, folder_path=f"{folder_save}/train")
print(f"Total records for train: {total_records_train}")


24/12/11 17:18:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/11 17:18:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/11 17:18:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/11 17:18:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/11 17:18:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/11 17:18:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/11 1

candle_type='Is_start_bullish_engulfing_candle'
directory_save='bullish_engulfing'
+------+---------------+-----------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+----------------------+--------------+--------------------+--------------------+-------------------+-------------------+--------------------+-------------------+-------------------+------------+---------------------+--------------------+--------------------+----------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+-------------------------------------+-----------------------+---------------------+--------------------------+---------------------------+--------------------+------------------+---------------------------+----------------+---------------+-----------------------+----------------------+----------------------+----------------

I0000 00:00:1733937519.008555    1313 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733937519.236866    1313 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733937519.236909    1313 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733937519.239355    1313 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733937519.239415    1313 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

candle_type='Is_start_bearish_engulfing_candle'
directory_save='bearish_engulfing'
+------+---------------+-----------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+---------------------+--------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------+---------------------+--------------------+--------------------+---------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------------------------------------+-----------------------+---------------------+--------------------------+---------------------------+--------------------+------------------+---------------------------+----------------+---------------+-----------------------+----------------------+----------------------+--------------------

In [ ]:
# For continue learning and test
is_run_for_continue_learning = input("Run for continue_learning? (y/n): ").lower()[0] == "y"
total_records_continue_learning = 0
if is_run_for_continue_learning:
    last_date_of_continue_learning = str(stock_df[Date].max().date()).replace("-", "_")
    folder_save_continue_learning = f"{folder_save}/continue_learning/to_{last_date_of_continue_learning}"
    os.makedirs(folder_save_continue_learning, exist_ok=True)
    
    query = generate_query(generate_include_month_query_for_test(month_test_in_year), candle_type_and_directory_save)
    total_records_continue_learning = save_dataset(query, candle_type_and_directory_save, folder_path=f"{folder_save_continue_learning}")
print(f"Total records for continue_learning: {total_records_continue_learning}")


Total records for continue learning: 0


In [ ]:
month_test_in_year_copy = month_test_in_year

In [ ]:
config = {
    "month_test_in_year": month_test_in_year_copy,
    "candle_type_and_directory_save": candle_type_and_directory_save,
    "folder_save": folder_save,
    "figscale": figscale,
    "previous_days": previous_days,
    "next_days": next_days,
    "days_result": days_result,
    "file_csv": file_csv,
    
    "indicator": indicator,
    "total_records_train": total_records_train,
    "total_records_test": total_records_test,
}

In [ ]:
config

{'month_test_in_year': {2012: [0],
  2013: [0],
  2014: [0],
  2015: [0],
  2016: [0],
  2017: [0],
  2018: [0],
  2019: [0],
  2020: [0],
  2021: [0],
  2022: [0],
  2023: [0],
  2024: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]},
 'candle_type_and_directory_save': {'Is_start_bullish_engulfing_candle': 'bullish_engulfing',
  'Is_start_bearish_engulfing_candle': 'bearish_engulfing',
  'Is_star_doji_candle': 'star_doji',
  'Is_dragonfly_doji_candle': 'dragonfly_doji',
  'Is_gravestone_doji_candle': 'gravestone_doji',
  'Is_start_morning_star_candle': 'morning_star',
  'Is_start_evening_star_candle': 'evening_star',
  'Is_hammer_candle': 'hammer',
  'Is_inverted_hammer_candle': 'inverted_hammer',
  'Is_start_harami_down_candle': 'harami_down',
  'Is_start_harami_up_candle': 'harami_up',
  'Is_start_tweezer_top_candle': 'tweezer_top',
  'Is_start_tweezer_bottom_candle': 'tweezer_bottom',
  'Is_marubozu_candle': 'marubozu',
  'Is_spin_candle': 'spin',
  'Is_start_next_3_day_up_candle': 'next_3

In [ ]:
if is_run_for_continue_learning:
    with open(f"{folder_save_continue_learning}/config.json", "w") as f:
        config["last_date_of_continue_learning"] = last_date_of_continue_learning
        config["total_records_continue_learning"] = total_records_continue_learning
        json.dump(config, f, indent=4)

In [ ]:
if is_run_for_test or is_run_for_train:
    with open(f"{folder_save}/config.json", "w") as f:
        json.dump(config, f, indent=4)